In [1]:
import numpy as np
import random
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler


#Store Data Variables
import json
with open('feature_data.json', 'r') as f:
 features = json.load(f)

from scipy.io import loadmat
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()
labels = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
filelist = loadmat('cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
camId = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()

In [2]:
train_idxs

array([    1,     2,     3, ..., 14094, 14095, 14096], dtype=uint16)

In [3]:
query_idxs

array([   22,    27,    49, ..., 14043, 14059, 14063], dtype=uint16)

In [4]:
labels

array([   1,    1,    1, ..., 1467, 1467, 1467], dtype=uint16)

In [5]:
gallery_idxs

array([   21,    23,    24, ..., 14062, 14064, 14065], dtype=uint16)

In [6]:
filelist

array([array(['1_001_1_01.png'], dtype='<U14'),
       array(['1_001_1_02.png'], dtype='<U14'),
       array(['1_001_1_03.png'], dtype='<U14'), ...,
       array(['5_049_2_08.png'], dtype='<U14'),
       array(['5_049_2_09.png'], dtype='<U14'),
       array(['5_049_2_10.png'], dtype='<U14')], dtype=object)

In [7]:
camId

array([1, 1, 1, ..., 2, 2, 2], dtype=uint8)

In [8]:
#grid searc cv against k, distance metric and transformation/normalization - use only query and gallery for this part 

In [9]:
#scaler = StandardScaler()
print(np.array(features))
#features = scaler.fit_transform(features)
X = np.array(features)
print(X)
y = np.array(labels)
filelist = np.array(filelist)
camId = np.array(camId)

[[0.13791114 1.12505555 0.05404324 ... 0.10747871 0.04081609 0.68009406]
 [0.02507781 0.9057585  0.00346441 ... 0.16763815 0.14764351 0.2700713 ]
 [0.09923808 1.09122825 0.01889733 ... 0.07981343 0.04958951 0.37923682]
 ...
 [0.50913167 2.1067946  1.12224829 ... 0.78767842 1.68007588 0.00259321]
 [0.44684452 1.87411916 1.51910186 ... 1.02090526 1.58616257 0.10876646]
 [0.46024311 2.31024432 1.20531154 ... 0.62753201 1.22394812 0.        ]]
[[0.13791114 1.12505555 0.05404324 ... 0.10747871 0.04081609 0.68009406]
 [0.02507781 0.9057585  0.00346441 ... 0.16763815 0.14764351 0.2700713 ]
 [0.09923808 1.09122825 0.01889733 ... 0.07981343 0.04958951 0.37923682]
 ...
 [0.50913167 2.1067946  1.12224829 ... 0.78767842 1.68007588 0.00259321]
 [0.44684452 1.87411916 1.51910186 ... 1.02090526 1.58616257 0.10876646]
 [0.46024311 2.31024432 1.20531154 ... 0.62753201 1.22394812 0.        ]]


In [10]:
mask_train = np.array(train_idxs).ravel()
mask_query = np.array(query_idxs).ravel()
mask_gallery = np.array(gallery_idxs).ravel()

mask_train = np.subtract(mask_train, 1)
mask_query = np.subtract(mask_query, 1)
mask_gallery = np.subtract(mask_gallery, 1)


X_train, X_query, X_gallery = X[mask_train, :], X[mask_query, :], X[mask_gallery, :]
y_train, y_query, y_gallery = y[mask_train], y[mask_query], y[mask_gallery]
filelist_train, filelist_query, filelist_gallery = filelist[mask_train], filelist[mask_query], filelist[mask_gallery]
camId_train, camId_query, camId_gallery = camId[mask_train], camId[mask_query], camId[mask_gallery]

In [11]:
import pandas as pd 
import numpy as np

from keras import layers, optimizers, regularizers
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.models import Sequential

from keras.utils import plot_model
#from kt_utils import *
import keras.backend as K

from sklearn import preprocessing, model_selection 
from keras.wrappers.scikit_learn import KerasRegressor

C:\Users\x201\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
# create model
model = Sequential()
# layer 1
model.add(Dense(6144, input_dim=6144, activation='relu', kernel_initializer='normal'))
#layer 2
model.add(Dense(128, activation='relu', kernel_initializer='normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
#layer 3
model.add(Dense(128, activation='relu',kernel_initializer='normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
#layer 4
model.add(Dense(2, activation='softmax'))
# Compile model
from keras import metrics
optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer, loss='categorical_crossentropy', metrics=[metrics.mae, metrics.categorical_accuracy]) 

In [18]:
X_train_pairs = []
y_train_pair_lables = []
for Xtnn, ytnn in zip(X_train, y_train):
    
    randindex = random.randint(0, 100)
    Xtnn2 = X_train[randindex]
    ytnn2 = y_train[randindex]
    
    randindex = random.randint(0, 100)
    Xtnn3 = X_train[randindex]
    ytnn3 = y_train[randindex]                 
    
    if((ytnn == ytnn2) and (ytnn == ytnn2)):
        y_train_pair_lables.append((1,1))
    if((ytnn == ytnn2) and (ytnn != ytnn2)):
        y_train_pair_lables.append((1,0))
    if((ytnn != ytnn2) and (ytnn == ytnn2)):
        y_train_pair_lables.append((0,1))
    if((ytnn != ytnn2) and (ytnn != ytnn2)):
        y_train_pair_lables.append((0,0))
    
    Xconcat = np.concatenate((Xtnn,Xtnn2,Xtnn3), axis = None)
    X_train_pairs.append(Xconcat)

y_train_pair_lables = np.array(y_train_pair_lables)
X_train_pairs = np.array(X_train_pairs)
model.fit(X_train_pairs, y_train_pair_lables, epochs=20)
model.metrics_names

Epoch 1/20
7368/7368 [==============================] - 323s 44ms/step - loss: 0.0054 - mean_absolute_error: 0.5000 - categorical_accuracy: 0.5060
Epoch 2/20
7368/7368 [==============================] - 320s 43ms/step - loss: 0.0065 - mean_absolute_error: 0.5000 - categorical_accuracy: 0.5119
Epoch 3/20
7368/7368 [==============================] - 309s 42ms/step - loss: 0.0082 - mean_absolute_error: 0.5000 - categorical_accuracy: 0.5132
Epoch 4/20
7368/7368 [==============================] - 310s 42ms/step - loss: 0.0054 - mean_absolute_error: 0.5000 - categorical_accuracy: 0.5358
Epoch 5/20
7368/7368 [==============================] - 305s 41ms/step - loss: 0.0045 - mean_absolute_error: 0.5000 - categorical_accuracy: 0.5193
Epoch 6/20
7368/7368 [==============================] - 302s 41ms/step - loss: 0.0053 - mean_absolute_error: 0.5000 - categorical_accuracy: 0.5209
Epoch 7/20
7368/7368 [==============================] - 307s 42ms/step - loss: 0.0068 - mean_absolute_error: 0.5000 - 

['loss', 'mean_absolute_error', 'categorical_accuracy']

In [22]:
randindexes = random.sample(range(0,7367), 1000)
X_val = []
y_val = []
camId_val = []
index_to_del = []
for i in randindexes:
    X_val.append(X_train[i])
    y_val.append(y_train[i])
    camId_val.append(camId_train[i])

In [23]:
X_val

[array([0.30778775, 0.81063157, 0.05659593, ..., 0.0764853 , 1.84792924,
        0.65402418]),
 array([0.51032758, 0.87834823, 0.49241096, ..., 0.11018689, 0.31702337,
        0.04707571]),
 array([0.6621483 , 0.14447016, 0.31392729, ..., 0.35745531, 0.        ,
        0.        ]),
 array([0.02184707, 0.13652201, 0.        , ..., 0.31993243, 0.40597931,
        0.01362633]),
 array([1.05512261, 0.0523683 , 0.99532521, ..., 0.52851987, 1.99223125,
        1.58189964]),
 array([0.09573962, 0.02075879, 1.96524918, ..., 0.98853785, 0.04308273,
        0.36726871]),
 array([0.07753071, 0.07565606, 0.32280701, ..., 0.04507868, 0.33593744,
        0.34430826]),
 array([0.09714381, 0.62400472, 0.        , ..., 0.18343399, 0.125485  ,
        0.14206697]),
 array([0.03480967, 0.        , 0.        , ..., 0.11333936, 0.74301052,
        0.08329731]),
 array([3.59794237e-02, 3.35644126e-01, 5.12825429e-01, ...,
        1.01215276e-03, 1.30028272e+00, 3.09080273e-01]),
 array([0.20034507, 0.1056

In [24]:
def get_acc_score(y_valid, y_q, tot_label_occur):
    recall = 0
    true_positives = 0
    
    k = 0
    
    max_rank = 30
    
    rank_A = np.zeros(max_rank)
    AP_arr = np.zeros(11)
    
    while (recall < 1) or (k < max_rank):
        if (y_valid[k] == y_q):
            
            true_positives = true_positives + 1
            recall = true_positives/tot_label_occur
            precision = true_positives/(k+1)
            
            AP_arr[round((recall-0.05)*10)] = precision
            
            for n in range (k, max_rank):
                rank_A[n] = 1
            
        k = k+1
        
    max_precision = 0
    for i in range(10, -1, -1):
        max_precision = max(max_precision, AP_arr[i])
        AP_arr[i] = max_precision
    
    AP_ = AP_arr.sum()/11
    
    return AP_, rank_A

In [31]:
from scipy.spatial import distance

def evaluate_metric(X_query, camId_query, y_query, X_gallery, camId_gallery, y_gallery, parameters = None):

    for val, camId_v, y_v  in zip(X_val, camId_val, y_val):
        v_v_dists = []
        y_valid = []
        for val2, camId_v2, y_v2  in zip(X_val, camId_val, y_val):
            if (((camId_v == camId_v2) and (y_v == y_v2)) or (val == val2)):
                continue
            else:
                stay = True
                while (stay):
                    randindex = random.randint(0, 100)
                    if((X_val[randindex] != val) and (X_val[randindex] != val2)):
                        val3 = X_val[randindex]
                        y_v3 = y_val[randindex]
                        stay = False
                dist = model.predict(np.concatenate((val, val2, val3)).reshape((1,6144)))[0][1]
                print(dist)

                v_v_dists.append(dist)
                y_valid.append(y_v3)

        tot_label_occur = y_valid.count(y_v1)
        v_v_dists = np.array(v_v_dists)
        y_valid = np.array(y_valid)

    
    _indexes = np.argsort(v_v_dists)

    # Sorted distances and labels
    v_v_dists, y_valid = v_v_dists[_indexes], y_valid[_indexes]

    AP_, rank_A = get_acc_score(y_valid, y_v1, tot_label_occur)

    AP.append(AP_)

    rank_accuracies.append(rank_A) 

    #if q  > 5:
    #    break
        #q = q+1

    rank_accuracies = np.array(rank_accuracies)

    total = rank_accuracies.shape[0]
    rank_accuracies = rank_accuracies.sum(axis = 0)
    rank_accuracies = np.divide(rank_accuracies, total)

    i = 0
    print ('Accuracies by Rank:')
    while i < rank_accuracies.shape[0]:
        print('Rank ', i+1, ' = %.2f%%' % (rank_accuracies[i] * 100), '\t',
              'Rank ', i+2, ' = %.2f%%' % (rank_accuracies[i+1] * 100), '\t',
              'Rank ', i+3, ' = %.2f%%' % (rank_accuracies[i+2] * 100), '\t',
              'Rank ', i+4, ' = %.2f%%' % (rank_accuracies[i+3] * 100), '\t',
              'Rank ', i+5, ' = %.2f%%' % (rank_accuracies[i+4] * 100))
        i = i+5

    AP = np.array(AP)

    mAP = AP.sum()/AP.shape[0]
    print('mAP = %.2f%%' % (mAP * 100))

    return rank_accuracies, mAP

In [32]:
rank_accuracies, mAP = evaluate_metric(X_val, camId_val, y_val,
                                       X_val, camId_val, y_val,
                                       parameters = None)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()